# ASSIGNMENT 4 - RUNNING CONVOLUTION

In [1]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import Audio
import math

### Load the dry signal and the impulse response

##### make sure they are mono and have the same sample rate

### Running convolution

##### You can use functions (and code) done in class to perform direct/fast convolutions within the Running Convolution algorithm. However, note that the assignment consists on the running convolution part.

In [2]:
dry, fs1 = sf.read('mono_sweep_original.wav')
ir, fs2 = sf.read('garage_sweep_IR_longer.wav')
snare,fs3 = sf.read('Snare.wav')
snare_m = snare[:,0]

#Audio(ir,rate=fs2)

In [3]:
def dir_conv(h, x): #Direct Convolution Function
    length = len(x) + len(h) - 1
    place_h = np.zeros(length)

    for i in range(0,len(h)):
        st = i
        en = i + len(x)
        place_h[st:en] = place_h[st:en] + h[i] * x
        final = place_h
    return (final)

def fast_conv(h, x): #Fast Convolution Function
    #Zero padding snare:
    num_zeros_x = np.zeros(len(h)-1)
    x_z = np.hstack([x,num_zeros_x])
    x_fft = np.fft.fft(x_z)

    #Zero padding IR:
    num_zeros_h = np.zeros(len(x)-1)
    h_z = np.hstack([h,num_zeros_h])
    h_fft = np.fft.fft(h_z)

    #Multiplying:
    fast_conv = x_fft * h_fft
    #Inverse Fourier transform
    fast_conv_ifft = np.fft.ifft(fast_conv)
    #Removing imaginary part
    fast_conv_ifft_a = np.real(fast_conv_ifft)
    final = fast_conv_ifft_a
    return (final)
        

def convolver (ir_sig, sig, conv_method, seg_len):
    
    # INPUTS
    # ir:  array containing the impulse response
    # sig: array containing a signal
    # conv_method: convolution method:
    #               1: direct convolution
    #               2: fast convolution
    # seg_len: length of input segments to be convolved with impulse response 

    # OUTPUTS:
    # out_sig:         Post-convolution output signal vector
        
    h = ir_sig
    x = sig 
    final = [] 
    total_segs = int(np.ceil(len(x)/seg_len))
    pad_len = len(h) - 1
    out_sig = np.zeros(len(x)+pad_len)
    
    for a in range(0, total_segs): 
        start_point = a * seg_len
        end_point = start_point + seg_len
        if (end_point>len(x)):
            end_point = len(x)
        buffer_signal = x[start_point:end_point]
        if (conv_method == "direct"): # direct convolution, method from class
            conv_output = dir_conv(ir_sig, buffer_signal)
            
        if (conv_method == "fast"): # fast convolution, method from class
            conv_output = fast_conv(ir_sig, buffer_signal)
        endpoint_conv = end_point + pad_len
        out_sig[start_point: endpoint_conv] += conv_output #+= to manage overlap
        
    return out_sig
    

### Use your function to convolve Snare.wav against your IR

In [7]:
fs = fs2
final = convolver (ir, snare_m, "fast", 1024) 
Audio(final, rate=fs)


[ 1.45006925e-04 -6.43088482e-04 -2.19716970e-03 ... -4.65661287e-09
 -9.31322575e-10 -4.65661287e-09]


### Normalize the result to avoid clipping

In [8]:
final_norm = (final - np.mean(final)) / np.std (final)
Audio(final_norm, rate=fs)


[ 8.23670502e-04 -3.65284680e-03 -1.24802951e-02 ... -2.01926258e-08
  9.67663052e-10 -2.01926258e-08]


### Export it

In [6]:
sf.write("verbsnare.wav", final_norm, fs)
print("done :)")

done :)
